In [ ]:
#사진
import cv2
import numpy as np
import openvino as ov
import os
from deepface import DeepFace
import tempfile
import shutil

core = ov.Core()
folder_path = 'class01'
img_path = 'asd.jpg'

# 얼굴 감지 모델
face_detection_model = "models/face-detection-adas-0001.xml"
face_detection_compiled_model = core.compile_model(model=face_detection_model, device_name="CPU")
face_input_layer = face_detection_compiled_model.input(0)
face_output_layer = face_detection_compiled_model.output(0)
face_input_shape = face_input_layer.shape

# 얼굴 랜드마크 모델
landmark_model = "models/landmarks-regression-retail-0009.xml"
landmark_compiled_model = core.compile_model(model=landmark_model, device_name="CPU")
landmark_input_layer = landmark_compiled_model.input(0)
landmark_output_layer = landmark_compiled_model.output(0)
landmark_input_shape = landmark_input_layer.shape

image_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
image_files.sort()

def preprocess_image(image, input_shape):
    if image is None:
        raise ValueError("이미지 없음")
    if len(image.shape) == 3 and image.shape[2] == 4:
        image = cv2.cvtColor(image, cv2.COLOR_BGRA2BGR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (input_shape[3], input_shape[2]))
    image = np.transpose(image, (2, 0, 1))
    image = np.expand_dims(image, axis=0)
    image = image.astype(np.float32)
    return image

def detect_faces(image):
    preprocessed_image = preprocess_image(image, face_input_shape)
    result = face_detection_compiled_model([preprocessed_image])[face_output_layer]
    return result[0][0]

def get_landmarks(image):
    preprocessed_image = preprocess_image(image, landmark_input_shape)
    result = landmark_compiled_model([preprocessed_image])[landmark_output_layer]
    return result.reshape(-1, 2)

def is_frontal_face(landmarks, threshold=0.25):
    left_eye = landmarks[0]
    right_eye = landmarks[1]
    nose_tip = landmarks[2]
    left_mouth = landmarks[3]
    right_mouth = landmarks[4]
    eye_distance = np.linalg.norm(left_eye - right_eye)
    nose_to_left_eye = np.linalg.norm(nose_tip - left_eye)
    nose_to_right_eye = np.linalg.norm(nose_tip - right_eye)
    mouth_width = np.linalg.norm(left_mouth - right_mouth)
    if abs(nose_to_left_eye - nose_to_right_eye) < threshold * eye_distance and abs(nose_to_left_eye - mouth_width) < threshold * eye_distance:
        return True
    return False

def save_temp_image(image):
    temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.jpg')
    temp_file_path = temp_file.name
    cv2.imwrite(temp_file_path, image)
    temp_file.close()
    return temp_file_path

frame = cv2.imread(img_path)
faces = detect_faces(frame)
for face in faces:
    confidence = face[2]
    if confidence > 0.5:
        xmin = int(face[3] * frame.shape[1])
        ymin = int(face[4] * frame.shape[0])
        xmax = int(face[5] * frame.shape[1])
        ymax = int(face[6] * frame.shape[0])
        if xmin < 0 or ymin < 0 or xmax > frame.shape[1] or ymax > frame.shape[0]:
            continue
        
        face_image = frame[ymin:ymax, xmin:xmax]
        if face_image.size == 0:
            continue
        
        landmarks = get_landmarks(face_image)
        for landmark in landmarks:
            cx, cy = int(landmark[0] * face_image.shape[1]), int(landmark[1] * face_image.shape[0])
            cx += xmin
            cy += ymin
            cv2.circle(frame, (cx, cy), 2, (0, 255, 255), -1, cv2.LINE_AA)
        if is_frontal_face(landmarks):
            temp_face_image_path = save_temp_image(face_image)
            verified = [10]
            for num, img_path in enumerate(image_files):
                try:
                    result = DeepFace.verify(img_path, temp_face_image_path)
                except:
                    verified = [10]
                    break
                if result["verified"] and verified[0] > result['distance']:
                    verified = [result['distance'], num, img_path]
            if verified == [10]:
                cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (0, 0, 255), 1, cv2.LINE_AA)
                cv2.putText(frame, "Unknown", (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
            else:
                cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (0, 255, 0), 1, cv2.LINE_AA)
                cv2.putText(frame, f"name: {verified[2][0: int(verified[2].find('.'))]}", (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)
            os.remove(temp_face_image_path)
        else:
            cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (255, 255, 255), 1, cv2.LINE_AA)
            cv2.putText(frame, "not frontal", (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
cv2.imshow('Webcam', frame)
key = cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
#라이브용
import cv2
import numpy as np
import openvino as ov
import os
from deepface import DeepFace
import tempfile
import shutil

core = ov.Core()
folder_path = 'class01'

# 얼굴 감지 모델
face_detection_model = "models/face-detection-adas-0001.xml"
face_detection_compiled_model = core.compile_model(model=face_detection_model, device_name="CPU")
face_input_layer = face_detection_compiled_model.input(0)
face_output_layer = face_detection_compiled_model.output(0)
face_input_shape = face_input_layer.shape

# 얼굴 랜드마크 모델
landmark_model = "models/landmarks-regression-retail-0009.xml"
landmark_compiled_model = core.compile_model(model=landmark_model, device_name="CPU")
landmark_input_layer = landmark_compiled_model.input(0)
landmark_output_layer = landmark_compiled_model.output(0)
landmark_input_shape = landmark_input_layer.shape

image_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
image_files.sort()

def preprocess_image(image, input_shape):
    if image is None:
        raise ValueError("이미지 없음")
    if len(image.shape) == 3 and image.shape[2] == 4:
        image = cv2.cvtColor(image, cv2.COLOR_BGRA2BGR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (input_shape[3], input_shape[2]))
    image = np.transpose(image, (2, 0, 1))
    image = np.expand_dims(image, axis=0)
    image = image.astype(np.float32)
    return image

def detect_faces(image):
    preprocessed_image = preprocess_image(image, face_input_shape)
    result = face_detection_compiled_model([preprocessed_image])[face_output_layer]
    return result[0][0]

def get_landmarks(image):
    preprocessed_image = preprocess_image(image, landmark_input_shape)
    result = landmark_compiled_model([preprocessed_image])[landmark_output_layer]
    return result.reshape(-1, 2)

def is_frontal_face(landmarks, threshold=0.25):
    left_eye = landmarks[0]
    right_eye = landmarks[1]
    nose_tip = landmarks[2]
    left_mouth = landmarks[3]
    right_mouth = landmarks[4]
    eye_distance = np.linalg.norm(left_eye - right_eye)
    nose_to_left_eye = np.linalg.norm(nose_tip - left_eye)
    nose_to_right_eye = np.linalg.norm(nose_tip - right_eye)
    mouth_width = np.linalg.norm(left_mouth - right_mouth)
    if abs(nose_to_left_eye - nose_to_right_eye) < threshold * eye_distance and abs(nose_to_left_eye - mouth_width) < threshold * eye_distance:
        return True
    return False

def save_temp_image(image):
    temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.jpg')
    temp_file_path = temp_file.name
    cv2.imwrite(temp_file_path, image)
    temp_file.close()
    return temp_file_path

cap = cv2.VideoCapture(0)
check = ["not attended"] * len(image_files)

while True:
    ret, frame = cap.read()
    if not ret:
        print("noCam")
        break
    
    faces = detect_faces(frame)
    for face in faces:
        confidence = face[2]
        if confidence > 0.5:
            xmin = int(face[3] * frame.shape[1])
            ymin = int(face[4] * frame.shape[0])
            xmax = int(face[5] * frame.shape[1])
            ymax = int(face[6] * frame.shape[0])
            if xmin < 0 or ymin < 0 or xmax > frame.shape[1] or ymax > frame.shape[0]:
                continue
            
            face_image = frame[ymin:ymax, xmin:xmax]
            if face_image.size == 0:
                continue
            
            landmarks = get_landmarks(face_image)
            for landmark in landmarks:
                cx, cy = int(landmark[0] * face_image.shape[1]), int(landmark[1] * face_image.shape[0])
                cx += xmin
                cy += ymin
                cv2.circle(frame, (cx, cy), 2, (0, 255, 255), -1, cv2.LINE_AA)
            if is_frontal_face(landmarks):
                temp_face_image_path = save_temp_image(face_image)
                verified = [10]
                for num, img_path in enumerate(image_files):
                    try:
                        result = DeepFace.verify(img_path, temp_face_image_path)
                    except:
                        verified = [10]
                        break
                    if result["verified"] and verified[0] > result['distance']:
                        verified = [result['distance'], num, img_path]
                if verified == [10]:
                    cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (0, 0, 255), 1, cv2.LINE_AA)
                    cv2.putText(frame, "Unknown", (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                else:
                    cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (0, 255, 0), 1, cv2.LINE_AA)
                    cv2.putText(frame, f"name: {verified[2][0: int(verified[2].find('.'))]}", (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)
                    check[verified[1]] = "attend"
                os.remove(temp_face_image_path)
            else:
                cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (255, 255, 255), 1, cv2.LINE_AA)
                cv2.putText(frame, "not frontal", (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
    y = 0
    for num, img_path in enumerate(image_files):
        y += 20
        cv2.putText(frame, f"{img_path[0: int(img_path.find('.'))]} : {check[num]}", (0, 40 + y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
    cv2.imshow('Webcam', frame)
    key = cv2.waitKey(1)
    if key == 27:
        break
cap.release()
cv2.destroyAllWindows()